In [ ]:
#!pip install aeon
#!pip install tsfresh

In [1]:
import pandas as pd
import numpy as np
import aeon
import tsfresh
import os
import math
from sklearn import svm

In [ ]:
from aeon.datasets import load_classification
from aeon.datasets.tsc_data_lists import univariate_equal_length, univariate
from aeon.classification.distance_based import KNeighborsTimeSeriesClassifier, ElasticEnsemble
from sklearn.metrics import accuracy_score

In [ ]:
univariate_list = list(univariate_equal_length)
univariate_list.sort()

In [ ]:
accuracy_data = []
for dataset_name in univariate_list:
    train, train_labels = load_classification(dataset_name, split='TRAIN')
    test, test_labels = load_classification(dataset_name, split='test')

    xtrain = train.reshape(train.shape[0], -1)
    xtest = test.reshape(test.shape[0], -1)

    EE = ElasticEnsemble(proportion_of_param_options=0.5,
                          proportion_train_in_param_finding=0.5,
                          proportion_train_for_test=0.3,
                          n_jobs=-1,
                          random_state=42,
                          majority_vote=True)

    EE.fit(xtrain, train_labels)
    y_hat = EE.predict(xtest)
    acc = accuracy_score(y_hat, test_labels)

    accuracy_data.append({'Dataset Name': dataset_name, 'Accuracy': acc})

accuracy_df = pd.DataFrame(accuracy_data)


In [ ]:
accuracy_df.to_parquet('model_acc_1nn_raw.parquet', index=False)